# TP3 - SVM
### Estudiante: Francisco Javier Piqueras Martínez
### Ejercicio de Clasificación de los datos AirBnb

Pasos:
- 1. Estudio estadístico y limpieza de datos
- 2. Clasificación sobre el campo room_type usando SVC y LinearSVC
- 3. Afinación con de hiperparámetros
- 4. Resultado

In [ ]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score, GridSearchCV
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import confusion_matrix

In [ ]:
airbnb_data = pd.read_csv(os.path.join("data","airbnb.csv"))

In [ ]:
airbnb_data.head()

### 1. Estudio estadístico y limpieza de datos

En primer lugar observamos los distintos valores de la variable `room_type`

In [ ]:
print(airbnb_data['room_type'].unique())

Y vemos el balanceo de las mismas:

In [ ]:
airbnb_data.groupby("room_type").size()

A la hora de entrenar el modelo, es conveniente saber si las clases están balanceadas o no. En este caso, comprobamos que no lo están.

En primer lugar, vamos a eliminar variables que hagan ruido, como es el caso de neighbourhood. Esta, nos da la localización de la vivienda, al igual que neighbourhood group. Por experiencia en la realización del trabajo prático 1, el modelo funciona mejor con neighbourhood_group que con neighbourhood, por lo que vamos a dejar la primera y a eliminar la segunda.

In [ ]:
airbnb = airbnb_data.drop("neighbourhood", 1, inplace=False)

Ahora, vamos a echarle un ojo al resto de variables (tipos, cantidad, etc):

In [ ]:
airbnb.info()

Como se puede observar, todas las variables tienen 13321, por lo que no hay ningún valor faltante.

A continuación, separamos el dataset creando el training set (80%) y el text set (20%):

In [ ]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=38)

for id_train, id_test in split.split(airbnb, airbnb["room_type"]):
    train_set, test_set = airbnb.loc[id_train], airbnb.loc[id_test]

y_train = train_set["room_type"]
X_train = train_set.drop("room_type", 1, inplace=False) 
y_test = test_set["room_type"]
X_test = test_set.drop("room_type", 1, inplace=False)

A continuación, escalamos los valores numéricos y usamos OneHotEncoder para "dummificar" las variables categóricas. El motivo por el que uso OneHotEnconder y no OrdinalEncoder es porque usando el segundo, estamos asignando números a las categorías e indicando que cieetas categorias están relacionadas cuando no existe esta relación. Por lo que intuyo que el entrenamiento del modelo podría ser erróneo.

In [ ]:
airbnb_categorical = ["neighbourhood_group"]
airbnb_numerical = ["latitude","longitude","price","minimum_nights","number_of_reviews","reviews_per_month","calculated_host_listings_count","availability_365"]
airbnb_label = ["room_type"]

airbnb_col_transformer = ColumnTransformer([
    ("num_parser", StandardScaler(), airbnb_numerical),
    ("cat_parser", OneHotEncoder(), airbnb_categorical),
])

In [ ]:
airbnb_train_pipeline_svc = Pipeline([
    ("col_transformer", airbnb_col_transformer),
    ("train", SVC())
])

airbnb_train_pipeline_linearsvc = Pipeline([
    ("col_transformer", airbnb_col_transformer),
    ("train", LinearSVC())
])

### 2. Clasificación sobre el campo room_type usando SVC y LinearSVC

Seguidamente, clasificamos nuestros datos haciendo uso de SVC

In [ ]:
cross_val_score(airbnb_train_pipeline_svc, X_train, y_train, cv=5, scoring="accuracy", n_jobs=1)

Y bueno, nada mal. Ahora vamos a clasificarlos con LinearSVC:

In [ ]:
cross_val_score(airbnb_train_pipeline_linearsvc, X_train, y_train, cv=5, scoring="accuracy", n_jobs=1)

A primera vista, funciona mejor el SVC que el linearSVC.

### 3.  Afinación de hiperparámetros

Vamos a buscar los mejores valores gamma y c para nuestro modelo con kernel=RBF.

Para ello, vamos a realizar las dos búsquedas, una más abierta, y luego una reducida en la zona de mejores resultados para afinar los hiperparámetros.

In [ ]:
airbnb_train_pipeline_svc.get_params().keys()

In [ ]:
grid_first_params = {
    'train__C' : np.logspace(-15, 15, base=2, num=5),
    'train__gamma' : np.logspace(-15, 15, base=2, num=5),
    'train__kernel' : ['rbf'],
    'train__class_weight' : ['balanced']
}

grid_first_search = GridSearchCV(airbnb_train_pipeline_svc, grid_first_params, cv=10, n_jobs=-1)

grid_first_result = grid_first_search.fit(X_train, y_train)

#{'model__C': 32768.0,  -> 2^15
# 'model__class_weight': 'balanced',
# 'model__gamma': 0.005524271728019903,  -> 2^-7.5
# 'model__kernel': 'rbf'}

In [ ]:
grid_first_result.best_params_

Reajustamos C para y gamma para afinar un poco más los hiperparámetros

In [ ]:
grid_second_params = {
    'train__C' : np.logspace(10, 20, base=2, num=5),
    'train__gamma' : np.logspace(-10, 0, base=2, num=5),
    'train__kernel' : ['rbf'],
    'train__class_weight' : ['balanced']
}

grid_second_search = GridSearchCV(airbnb_train_pipeline_svc, grid_second_params, cv=10, n_jobs=-1, verbose=True)

grid_second_result = grid_second_search.fit(X_train, y_train)

In [ ]:
grid_second_result.best_params_

c = 2^15 gamma = 2^-5

Hacemos una última iteración:

In [ ]:
grid_third_params = {
    'train__C' : np.logspace(14.5, 15.5, base=2, num=5),
    'train__gamma' : np.logspace(-8, -4, base=2, num=5),
    'train__kernel' : ['rbf'],
    'train__class_weight' : ['balanced']
}

grid_third_search = GridSearchCV(airbnb_train_pipeline_svc, grid_first_params, cv=10, n_jobs=-1)

grid_third_result = grid_first_search.fit(X_train, y_train)



In [ ]:
grid_third_result.best_params_


c = 2^15.5

gamma = 2^-5

In [ ]:
airbnb_best_c = grid_third_result.best_params_['train__C']

In [ ]:
airbnb_best_gamma = grid_third_result.best_params_['train__gamma']

### 4. Resultado

Finalmente, creamos nuestro modelo con los hiperparámetros que hemos obtenido y vemos los resultados obtenidos:

In [ ]:
y_test = train_set["room_type"]
x_test = test_set.drop("room_type", 1, inplace=False)

In [ ]:
airbnb_train_pipeline_svc = Pipeline([
    ("col_transformer", airbnb_col_transformer),
    ("test", SVC(C=airbnb_best_c, gamma=airbnb_best_gamma))
])

airbnb_train_pipeline_svc.fit(X_train, y_train)
airbnb_train_pipeline_svc_prediction = airbnb_train_pipeline_svc.predict(X_test)

confusion_matrix(y_test, airbnb_train_pipeline_svc_prediction)